In [2]:
#as always, let's start by loading some required python modules
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi
import pandas as pd

# Loading the dataset into the repository
Total_data=pd.read_csv("../project_data/Bio165_InfluenzaAIGG_Assay.csv")

In [ ]:
# Saving columns for plotting: 
Time =  Total_data['day']

In [25]:
# Add new column for each data transformation: 

# NEED TO ADJUST AND FIGURE OUT WHAT THE DATA TRANSFORMATION WAS

Total_data["log2_Ctrl"]=np.log2(Total_data["Ctrl"])
Total_data["log2_Vax"]=np.log2(Total_data["Vax"])
Total_data["log2_IFNB"]=np.log2(Total_data["IFNB"])
Total_data["log2_Vax_IFNB"]=np.log2(Total_data["Vax_IFNB"])                                                                                                                                           

/Users/abubakrsangare/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
# Extracting each column in preparation for the log transformations
untrans_Ctrl = Total_data['Ctrl']  
untrans_Vax = Total_data['Vax']
untrans_IFNB = Total_data['IFNB']
untrans_Vax_IFNB = Total_data['Vax_IFNB']

In [24]:
# Group dataframe by the different columns––want to seperate IIV from LAIV conditions first
mean_Total_data = Total_data.groupby(["sti","day"], as_index=False).agg({"Ctrl": ['std','mean'],"Vax": ['std','mean'],"IFNB": ['std','mean'], "Vax_IFNB": ['std','mean']})
mean_Total_data.columns = mean_Total_data.columns.map('_'.join)
display(mean_Total_data)

,sti_,day_,Ctrl_std,Ctrl_mean,Vax_std,Vax_mean,IFNB_std,IFNB_mean,Vax_IFNB_std,Vax_IFNB_mean
0,IIV,3,1.267538,4.865931,2.652989,3.538942,2.566489,4.284288,2.760186,6.375904
1,IIV,7,1.167828,5.572559,2.346109,9.103023,3.822737,6.235578,1.078652,11.855246
2,IIV,10,1.131868,5.556852,1.522383,9.606592,4.376315,5.853949,1.506066,11.692530
3,LAIV,3,2.042530,4.201286,2.589663,6.626303,2.063915,5.656575,1.963120,7.967253
4,LAIV,7,3.249440,4.009043,1.712749,11.030848,2.782359,7.786935,1.134803,11.230518
5,LAIV,10,2.700973,4.033554,1.695154,11.149275,3.143026,7.475857,1.066370,11.192619


In [ ]:
# Doing the log transformation on the averaged data
nonave_Ctrl = np.log2(untrans_Ctrl)
nonave_Vax = np.log2(untrans_Vax)
nonave_IFNB = np.log2(untrans_IFNB)
nonave_Vax_IFNB = np.log2(untrans_Vax_IFNB)

In [ ]:
# Now, we generate the first plot(s): 

plt.scatter(x, y)
plt.xlabel('Ribosomal Protein')
plt.ylabel('Growth Rate')